# Part 1 - Setting up a subwallet for Alice

### Initialise the basewallet controller

In [ ]:
%autoawait
import time
import asyncio
import pprint

from aries_basic_controller.aries_controller import AriesAgentController

# Create a small utility to print json formatted outout more human-readable    
pp = pprint.PrettyPrinter(indent=4)

WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_BASE = ""

WEBHOOK_PORT = 8022
ADMIN_URL = "http://basewallet-agent:8021"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, multitenant=True)


### Creating a subwallet (again)

At the end of the previous notebook we have removed all subwallets. We have done so, in order to demonstrate how to achieve this. We'll need subwallets though so let's create a new one for Alice.

In [ ]:
## First let's create the payload

payload = {
  "image_url": "https://aries.ca/images/sample.png",
  "key_management_mode": "managed",
  "label": "Alice",
  "wallet_dispatch_type": "default",
  "wallet_key": "MySecretKey123",
  "wallet_name": "AlicesWallet",
  "wallet_type": "indy",
  "wallet_webhook_urls": [
    "http://localhost:8022/webhooks"
  ]
}

In [ ]:
## Now, we create the wallet on the agent 
response_alice = await agent_controller.multitenant.create_subwallet(payload)
pp.pprint(response_alice)

### Updating jwt_token of the agent controller

First we need to obtain the token from the wallet via the agent using the wallet id

In [ ]:
wallet_id_alice = response_alice['wallet_id']

print("Alice's wallet ID: " + wallet_id_alice)

In [ ]:
jwt_token_response_alice = await agent_controller.multitenant.get_subwallet_authtoken_by_id(wallet_id_alice)
jwt_token_alice = jwt_token_response_alice['token']
print("JWT Token: " + jwt_token_alice)

Now we can update the agent controller with the JWT Token

In [ ]:
agent_controller.update_jwt_token(jwt_token_alice)

In [ ]:
# res = await agent_controller.terminate()
# print(res)

In [ ]:
# agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
#                                        webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, multitenant=True, jwt_token=jwt_token_alice['token'])


Let's check it's really there

In [ ]:
print(agent_controller.jwt_token)

### Accept Invite From Mediator



In [ ]:
mediator_invitation = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': 'ad421292-5b26-4f92-aeb9-64fc0864673d', 'recipientKeys': ['CjF811R3iFTn3fjRmuv3eg9BE7eDZEfgheocVenXFEdp'], 'label': 'MEDIATOR', 'serviceEndpoint': 'https://97ea36fd4863.ngrok.io'}

In [ ]:
response = await agent_controller.connections.accept_connection(mediator_invitation)
pp.pprint(response)

In [ ]:
connection_id = response["connection_id"]
print(connection_id)

In [ ]:
response = await agent_controller.mediation.get_mediation_records()
print(response)
print('\n')
mediation_id = response[0]['mediation_id']

### Mediation requested

Notice in the output above that we have requested mediation so the `state` is now `requested`.

We can now head back over to [Bob](http://localhost:8888/notebooks/Bob/Part%202%20-%20Mediation.ipynb) to accept the request.

### Use Bob as our default mediator

Now that Bob has granted our request let's set him as our defautl mediator

In [ ]:
response = await agent_controller.mediation.set_default_mediator(mediation_id)
print(response)

### Set a default mediator

### Check whether there really is our default mediator

In [ ]:
response = await agent_controller.mediation.get_default_mediator()
print(response)

In [ ]:
await agent_controller.terminate()